In [24]:
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import numpy as np
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

df_day = pd.read_csv(config['Data_filename_day'], parse_dates=['Time'], index_col='Time')
# df_day.drop(columns=['Volume', 'Change', 'ChangePercent', 'Vwap'], inplace=True)
df_day.drop(columns=['Volume'], inplace=True)
# df_day['EMA10'] = ta.ema(df_day['Close'])
# df_day['EMA50'] = ta.ema(df_day['Close'], length=50)
# df_day.dropna(inplace=True)
# df_day['MTI'] = (df_day['EMA10'] - df_day['EMA50'])
# mti_green_mask = ((df_day['MTI'] > 0.1) & (df_day['Close'].shift(1) > df_day['Close'].shift(2)))
# df_day.loc[(df_day['MTI'] > 0.1),'COLOR'] = 'G'
# df_day.loc[df_day['MTI'] < -0.1,'COLOR'] = 'R'
# df_day.loc[(df_day['MTI'] < 0.1) & (df_day['MTI'] > -0.1),'COLOR'] = 'Y'
df_day['SMA20'] = ta.sma(df_day['Close'], length=20)
df_day['SMA50'] = ta.sma(df_day['Close'], length=50)
df_day.dropna(inplace=True)
df_day['Color'] = 'Y'
df_day.loc[df_day['SMA20'] < df_day['SMA50'],'Color'] = 'R'
df_day.loc[(df_day['SMA20'] > df_day['SMA50']) & (df_day['Close'] > df_day['SMA20']) & (df_day['SMA20'] > df_day['SMA20'].shift(7)),'Color'] = 'G'
# df_day.drop(columns=['SMA20', 'SMA50'], inplace=True)
# df_day

In [25]:
def chandelier_exit(df_day, length=22, mult=3.0, use_close=True):
        
    df = df_day.copy()
    
    # Calculate ATR
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # True Range calculation
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # ATR using Wilder's smoothing (EMA with alpha = 1/length)
    atr = tr.ewm(alpha=1/length, adjust=False).mean()
    atr_mult = mult * atr
    
    # Calculate long stop
    if use_close:
        highest = close.rolling(window=length).max()
    else:
        highest = high.rolling(window=length).max()
    
    long_stop = highest - atr_mult
    
    # Apply long stop logic: can only move up or stay same
    long_stop_prev = long_stop.shift(1)
    close_prev = close.shift(1)
    
    for i in range(1, len(long_stop)):
        if pd.notna(long_stop_prev.iloc[i]) and pd.notna(close_prev.iloc[i]):
            if close_prev.iloc[i] > long_stop_prev.iloc[i]:
                long_stop.iloc[i] = max(long_stop.iloc[i], long_stop_prev.iloc[i])
    
    # Calculate short stop
    # if use_close:
    #     lowest = close.rolling(window=length).min()
    # else:
    #     lowest = low.rolling(window=length).min()
    
    # short_stop = lowest + atr_mult
    
    # # Apply short stop logic: can only move down or stay same
    # short_stop_prev = short_stop.shift(1)
    
    # for i in range(1, len(short_stop)):
    #     if pd.notna(short_stop_prev.iloc[i]) and pd.notna(close_prev.iloc[i]):
    #         if close_prev.iloc[i] < short_stop_prev.iloc[i]:
    #             short_stop.iloc[i] = min(short_stop.iloc[i], short_stop_prev.iloc[i])
    
    # Determine direction
    # direction = pd.Series(index=df.index, dtype=int)
    # direction.iloc[0] = 1  # Start with long
    
    # for i in range(1, len(direction)):
    #     if pd.notna(short_stop_prev.iloc[i]) and close.iloc[i] > short_stop_prev.iloc[i]:
    #         direction.iloc[i] = 1
    #     elif pd.notna(long_stop_prev.iloc[i]) and close.iloc[i] < long_stop_prev.iloc[i]:
    #         direction.iloc[i] = -1
    #     else:
    #         direction.iloc[i] = direction.iloc[i-1]
    
    # Generate signals
    # direction_prev = direction.shift(1)
    # buy_signal = (direction == 1) & (direction_prev == -1)
    # sell_signal = (direction == -1) & (direction_prev == 1)
    
    # Add indicator columns to the original dataframe
    df['Ch_exit'] = long_stop
    # df['short_stop'] = short_stop
    # df['direction'] = direction
    # df['buy_signal'] = buy_signal
    # df['sell_signal'] = sell_signal
    # df['atr'] = atr
    
    return df

df_day = chandelier_exit(df_day, use_close=False)
df_day.dropna(inplace=True)
df_day.tail(50)

,Open,High,Low,Close,SMA20,SMA50,Color,Ch_exit
Time,,,,,,,,
2025-10-08 13:30:00+00:00,246.200,247.2600,245.0900,245.46,247.9995,224.7052,Y,241.014790
2025-10-09 13:30:00+00:00,245.060,245.5550,239.6801,242.21,248.0710,225.6006,Y,241.014790
2025-10-10 13:30:00+00:00,242.160,244.8400,236.6850,237.49,247.8765,226.4932,Y,240.926631
2025-10-13 13:30:00+00:00,241.030,244.9900,240.7500,244.64,247.5205,227.5870,Y,240.243759
2025-10-14 13:30:00+00:00,241.755,248.0250,241.1775,246.19,247.2590,228.5958,Y,240.243759
2025-10-15 13:30:00+00:00,248.085,252.7600,246.7900,251.71,247.3520,229.7236,G,240.058020
2025-10-16 13:30:00+00:00,252.465,257.5800,250.6700,251.88,247.3295,230.8228,Y,240.619084
2025-10-17 13:30:00+00:00,251.350,254.8800,248.5900,253.79,247.2570,231.9530,Y,240.619084
2025-10-20 13:30:00+00:00,255.230,257.8800,254.6000,257.02,247.4640,233.0516,Y,241.049475


In [ ]:
  
index_arr_day = df_day.index.to_numpy()
close_arr_day = df_day['Close'].to_numpy()
color_arr_day = df_day['Color'].to_numpy()
ch_exit_arr_day = df_day['Ch_exit'].to_numpy()

entry_arr_day = np.full(len(index_arr_day), False)
exit_arr_day = np.full(len(index_arr_day), False)
price_arr_day = np.full(len(index_arr_day), np.nan)

def backtest(df_day):

    for i in range(0, len(index_arr_day)):
        if not trade_is_open and color_arr_day[i] == 'G':
            trade_is_open = True
            entry_arr_day[i] = True
            close_price = ch_exit_arr_day[i]
            price_arr_day[i] = close_arr_day[i]
        elif trade_is_open and close_arr_day[i] <= close_price:
            trade_is_open = False
            exit_arr_day[i] = True
            close_price = np.nan
            price_arr_day[i] = close_arr_day[i]
        
    return df_day

df_day = backtest(df_day)

CH = 35.76873116629934
CL = 37.2475
CH = 35.690623131713245
CL = 36.921


In [27]:
pf = vbt.Portfolio.from_signals(
entries = entry_arr_day,
exits = exit_arr_day,
price = price_arr_day,
open = df_day["Open"],
close = close_arr_day,
size = config['Trade']['size'],
size_type = config['Trade']['size_type'],
init_cash = config['Initial_cash'],
freq = '1d'
)

In [28]:
pf.stats()

Start                         2016-03-31 13:30:00+00:00
End                           2025-12-17 14:30:00+00:00
Period                               2445 days 00:00:00
Start Value                                     10000.0
End Value                                    10259.2985
Total Return [%]                               2.592985
Benchmark Return [%]                          700.21478
Max Gross Exposure [%]                         3.146756
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.664748
Max Drawdown Duration                 546 days 00:00:00
Total Trades                                          2
Total Closed Trades                                   1
Total Open Trades                                     1
Open Trade PnL                                  261.139
Win Rate [%]                                        0.0
Best Trade [%]                                -4.941271
Worst Trade [%]                               -4

In [29]:
pf.trades.records_readable.tail(50)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,Open,1.0,2016-03-31 13:30:00+00:00,37.2475,0.0,2016-04-26 13:30:00+00:00,35.407,0.0,-1.8405,-0.049413,Long,Closed,0
1,1,Open,1.0,2016-07-26 13:30:00+00:00,36.9210,0.0,2025-12-17 14:30:00+00:00,298.060,0.0,261.1390,7.072912,Long,Open,1


In [30]:
from savetopdf import save_backtesting_results_to_pdf
file_path = config['Data_filename_day'].split('.')[0]
file_path= (f"{file_path}_{config['Days']}")
save_backtesting_results_to_pdf(pf, file_path)

CSV file with statistics 'data/NASDAQ:GOOG_day_[2015-12-17][2025-12-17]_insightsentry_0_stats.pdf' was created!
